In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Load API key
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You answer in one short sentence"),
    ("human", "{question}")
])

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"question": "What is LangChain?"}))

LangChain is a framework for developing applications powered by language models.


In [ ]:
#!pip install tavily-python

### Creating Multiple Tools

In [2]:
from langchain_core.tools import tool
from tavily import TavilyClient

#TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

@tool
def qa_tool(question: str) -> str:
    """Answer in one short sentence."""
    return chain.invoke({"question": question})

@tool
def summarize_tool(question: str) -> str:
    """Summarize input text."""
    return chain.invoke({"question": f"Summarize this:\n\n{text}"})

@tool
def search_tool(query: str) -> str:
    """Quick internet search (best single answer)."""
    results = tavily.search(query=query, max_results=1)
    return results["results"][0]["content"]


print("\n Simple QA tool:",qa_tool.invoke("What is LangChain?"))

text="LangChain is a framework for developing applications powered by large language models, often connecting them with external data and tools."

print("\n Summarizer Tool:",summarize_tool.invoke(text))

#search_text="Give the latest updtes on langchain?"

#print("\n Web search Tool:",search_tool.invoke(search_text))

search_text = "Give the latest updates on LangChain?"

print("\n Web search Tool:", search_tool.invoke({"query": search_text}))


 Simple QA tool: LangChain is a framework for developing applications powered by language models.

 Summarizer Tool: LangChain helps build applications using large language models by connecting them to external data and tools.

 Web search Tool: In this video, I cover LangChain's recent announcements about raising money at over a $1.25 valuation and the launch of LangChain and


### AGENT

In [3]:
from langchain.agents import initialize_agent, AgentType

tools = [qa_tool, summarize_tool,search_tool]
react_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # MAIN REACT TYPE
    verbose=True
)

print(react_agent.invoke({"input": "Summarise about langchain"}))


C:\Users\admin\AppData\Local\Temp\ipykernel_4384\320658337.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  react_agent = initialize_agent(




> Entering new AgentExecutor chain...
I need to summarize what Langchain is. I can use the summarize tool for this.
Action: summarize_tool
Action Input: Langchain
Observation: LangChain is a framework for building LLM-powered applications with external data and tools.
Thought:I now know the final answer
Final Answer: LangChain is a framework for building LLM-powered applications with external data and tools.

> Finished chain.
{'input': 'Summarise about langchain', 'output': 'LangChain is a framework for building LLM-powered applications with external data and tools.'}


### CONVERSATIONAL REACT DESCRIPTION

In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import create_tool_calling_agent, AgentExecutor

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#agent = create_tool_calling_agent(llm, tools)

react_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # MAIN REACT TYPE
    memory=memory,
    verbose=True
)


#agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

print(react_agent.invoke({"input": "what is langgraph?"}))
print(react_agent.invoke({"input": "Explain its ecosystem"}))   # remembers context


C:\Users\admin\AppData\Local\Temp\ipykernel_4384\712296217.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: search_tool
Action Input: langgraph definition
Observation: LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactions.
Thought:Do I need to use a tool? No
AI: LangGraph is a library within the LangChain ecosystem that provi

### CHAT CONVERSATIONAL REACT DESCRIPTION

In [5]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import create_tool_calling_agent, AgentExecutor

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#agent = create_tool_calling_agent(llm, tools)

react_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,  # MAIN REACT TYPE
    memory=memory,
    verbose=True
)


#agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

print(react_agent.invoke({"input": "summarize langchain in 2 lines then tell me who created it"}))




> Entering new AgentExecutor chain...
```json
{
    "action": "summarize_tool",
    "action_input": "Langchain is a framework for developing applications powered by language models. It enables the creation of applications that are data-aware and agentic."
}
```
Observation: LangChain is a framework for building LLM-powered applications.
Thought:```json
{
    "action": "search_tool",
    "action_input": "who created langchain"
}
```
Observation: Harrison Chase, the visionary co-founder of LangChain, has redefined how developers harness Language Learning Models (LLMs) through a revolutionary open-source framework. His work at LangChain not only empowers developers to build context-aware applications but also sets new standards for collaboration in the AI ecosystem. The idea for LangChain took root in 2022 when Chase recognized a recurring challenge: developers were struggling to build reliable LLM applications due to fragmented tools and insufficient abstractions. Beyond LangChain, Cha

### STRUCTURED CHAT ZERO SHOT REACT

In [6]:
from pydantic import BaseModel
from langchain.tools import StructuredTool

class TitleInput(BaseModel):
    topic: str
    tone:str="concise"

def title_tool_fn(topic:str,tone:str="concise") -> str:
    return f"{tone.title()} Title:{topic} in practice"

#Create the structured Tool
title_tool = StructuredTool.from_function(
    name = "TitleMaker",
    func = title_tool_fn,
    description = "Generate a title given a topic and an optional tone",
    args_schema = TitleInput
)

#Initialize agent
agent = initialize_agent(
    tools = [title_tool],
    llm = llm,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

# Run the agent

result = agent.run("Make a friendly title about langgraph tutorials")
print("Result:",result)

C:\Users\admin\AppData\Local\Temp\ipykernel_4384\1078109836.py:29: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run("Make a friendly title about langgraph tutorials")




> Entering new AgentExecutor chain...
```json
{
  "action": "TitleMaker",
  "action_input": {
    "topic": "langgraph tutorials",
    "tone": "friendly"
  }
}
```
Observation: Friendly Title:langgraph tutorials in practice
Thought:I have a title that is friendly.
Action:
```
{
  "action": "Final Answer",
  "action_input": "Friendly Title:langgraph tutorials in practice"
}
```

> Finished chain.
Result: Friendly Title:langgraph tutorials in practice


### RAG

In [8]:
#!pip install faiss-cpu

In [9]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

with open("sample.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)
texts = splitter.split_text(text_data)

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

vectorstore = FAISS.from_texts(texts,embeddings)

#vectorstore = Chroma.from_texts(texts, embedding=embeddings, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever()

print(retriever)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"     # simplest style: stuff retrieved chunks directly into prompt
)

query = "Summarize what LangChain is and its core purpose."
answer = qa_chain.invoke({"query": query})
print("Answer:\n", answer["result"])

tags=['FAISS', 'GoogleGenerativeAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000253FF688050> search_kwargs={}
Answer:
 LangChain is a framework created by Harrison Chase for building applications with LLMs. It supports RAG, agents, memory, tools, and more, and is commonly used in chatbots, document Q&A, and AI workflows.


### FileChatMessageHistory

In [10]:

from langchain_community.chat_message_histories import FileChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

FILE_PATH = "my_conversation_history.json"
if os.path.exists(FILE_PATH):
    # Optional: Delete the file to start clean for the demonstration
    os.remove(FILE_PATH) 

# Initialize the history object
chat_history = FileChatMessageHistory(file_path=FILE_PATH)

print(f"History initialized, saving to: {FILE_PATH}")
print("---")

# --- 2. Adding Messages ---

# Add messages using convenience methods
chat_history.add_user_message("Hello! My name is Alex.")
chat_history.add_ai_message("Hello Alex! I am Gemini, a helpful AI assistant. How can I assist you today?")

# Add a message using the BaseMessage object directly
follow_up_user = HumanMessage(content="What was my name again?")
chat_history.add_message(follow_up_user)

# --- 3. Retrieving and Displaying Messages ---
messages = chat_history.messages 

print(f"Total messages saved: {len(messages)}")
for i, message in enumerate(messages):
    print(f"[{i+1}] {message.type.capitalize()}: {message.content}")

print("---")


History initialized, saving to: my_conversation_history.json
---
Total messages saved: 3
[1] Human: Hello! My name is Alex.
[2] Ai: Hello Alex! I am Gemini, a helpful AI assistant. How can I assist you today?
[3] Human: What was my name again?
---
